In [4]:
import zipfile

# Unzip the downloaded file ( https://www.kaggle.com/lantian773030/pokemonclassification )
zip_ref = zipfile.ZipFile("archive.zip", "r")
zip_ref.extractall()
zip_ref.close()

In [6]:
# Delete the archive
!del /f archive.zip